In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
warnings.filterwarnings("ignore")

In [2]:
import os

print(os.listdir("data/"))

['.ipynb_checkpoints', 'creditcard.csv.zip', 'creditcardfraud.zip', 'data.zip', 'inpatientCharges.zip', 'Inpatient_prospective_payment.zip', 'ISF.zip', 'purchase_credit_card.zip', 'res_purchase_card.zip', 'S&P Benchmark.zip', 'Twillio_data.zip', 'us_regions.zip', 'Variables_Dictionary_2019.zip', 'XYZloan_default_adj.zip', 'XYZloan_default_selected_vars.zip']


### Introduction

So far, I've experimented and studied linear models for outlier detection (PCA), outlier ensemble techniques (Isolation forest). This week I want to experiment with a new type of model proximity outlier detection (Local Outlier Factor). 

In [3]:
dat = pd.read_csv('data/data.zip')
dat.head()

,Unnamed: 0,DRG Definition,Provider Id,Provider Name,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,...,ratio_average_covered_charges,ratio_average_total_payment,ratio_average_medicare_payments,ratio_oop_payment,ratio_oop_payment_ratio,ratio_payment_coverage,ratio_medicare_pct_paid,region_avg_medicare_paid,Medicare Paid Ratio,K Means 1
0,0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,AL - Dothan,91,32963.07,...,0.908445,1.037810,1.038763,1.033356,0.691641,1.128919,5.311252,0.823783,1.000954,0
1,1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85,...,0.376065,1.008540,1.080948,0.714704,1.274242,1.873207,4.211412,0.804224,1.069224,3
2,2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,FLORENCE,AL,35631,AL - Birmingham,24,37560.37,...,0.933470,0.947092,0.967369,0.864809,0.899420,0.708675,4.013431,0.804224,1.018959,0
3,3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28,...,0.347893,0.944062,0.896859,1.135614,1.133822,1.895443,3.732842,0.804224,0.947721,3
4,4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,ALABASTER,AL,35007,AL - Birmingham,18,31633.27,...,0.786167,0.986018,1.053739,0.711205,0.986225,0.876044,4.199175,0.804224,1.066118,0


In [4]:
dat.columns

Index(['Unnamed: 0', 'DRG Definition', 'Provider Id', 'Provider Name',
       'Provider City', 'Provider State', 'Provider Zip Code',
       'Hospital Referral Region Description', 'Total Discharges',
       'Average Covered Charges', 'Average Total Payments',
       'Average Medicare Payments', 'Out of Pocket Payment', 'Coverage Ratio',
       'Out of Pocket per discharge', 'Payment Covered Percentage',
       'Medicare % Paid', 'mean_discharge_per_drg_state_region',
       'mean_covered_charges_per_drg_state_region',
       'mean_total_payments_per_drg_state_region',
       'mean_medicare_payments_per_drg_state_region',
       'mean_trans_per_drg_state_region', 'mean_oop_drg_state_region',
       'mean_oop_per_discharge_drg_state_region',
       'mean_pmt_cov_pct_drg_state_region', 'mean_medicare_paid_state_region',
       'total_discharge_ratio', 'ratio_average_covered_charges',
       'ratio_average_total_payment', 'ratio_average_medicare_payments',
       'ratio_oop_payment', 'rat

In [5]:
dat = dat.drop(columns = ['Unnamed: 0','Provider City','K Means 1'], axis = 1)
dat.head()

,DRG Definition,Provider Id,Provider Name,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,...,total_discharge_ratio,ratio_average_covered_charges,ratio_average_total_payment,ratio_average_medicare_payments,ratio_oop_payment,ratio_oop_payment_ratio,ratio_payment_coverage,ratio_medicare_pct_paid,region_avg_medicare_paid,Medicare Paid Ratio
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,AL,36301,AL - Dothan,91,32963.07,5777.24,4763.73,...,1.338235,0.908445,1.037810,1.038763,1.033356,0.691641,1.128919,5.311252,0.823783,1.000954
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,AL,35957,AL - Birmingham,14,15131.85,5787.57,4976.71,...,0.509804,0.376065,1.008540,1.080948,0.714704,1.274242,1.873207,4.211412,0.804224,1.069224
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,AL,35631,AL - Birmingham,24,37560.37,5434.95,4453.79,...,0.873950,0.933470,0.947092,0.967369,0.864809,0.899420,0.708675,4.013431,0.804224,1.018959
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,AL,35235,AL - Birmingham,25,13998.28,5417.56,4129.16,...,0.910364,0.347893,0.944062,0.896859,1.135614,1.133822,1.895443,3.732842,0.804224,0.947721
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,AL,35007,AL - Birmingham,18,31633.27,5658.33,4851.44,...,0.655462,0.786167,0.986018,1.053739,0.711205,0.986225,0.876044,4.199175,0.804224,1.066118


In [6]:
# !pip install pyod

For this, I will take only the numerical columns:

In [7]:
# extracting numerical columns : 
num_dat = dat._get_numeric_data()
num_dat.columns

Index(['Provider Id', 'Provider Zip Code', 'Total Discharges',
       'Average Covered Charges', 'Average Total Payments',
       'Average Medicare Payments', 'Out of Pocket Payment', 'Coverage Ratio',
       'Out of Pocket per discharge', 'Payment Covered Percentage',
       'Medicare % Paid', 'mean_discharge_per_drg_state_region',
       'mean_covered_charges_per_drg_state_region',
       'mean_total_payments_per_drg_state_region',
       'mean_medicare_payments_per_drg_state_region',
       'mean_trans_per_drg_state_region', 'mean_oop_drg_state_region',
       'mean_oop_per_discharge_drg_state_region',
       'mean_pmt_cov_pct_drg_state_region', 'mean_medicare_paid_state_region',
       'total_discharge_ratio', 'ratio_average_covered_charges',
       'ratio_average_total_payment', 'ratio_average_medicare_payments',
       'ratio_oop_payment', 'ratio_oop_payment_ratio',
       'ratio_payment_coverage', 'ratio_medicare_pct_paid',
       'region_avg_medicare_paid', 'Medicare Paid Ratio

In [8]:
# deleting the unneeded columns: 
num_dat = num_dat.drop(['Provider Id', 'Provider Zip Code'], axis = 1)
num_dat.head()

,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,Out of Pocket Payment,Coverage Ratio,Out of Pocket per discharge,Payment Covered Percentage,Medicare % Paid,mean_discharge_per_drg_state_region,...,total_discharge_ratio,ratio_average_covered_charges,ratio_average_total_payment,ratio_average_medicare_payments,ratio_oop_payment,ratio_oop_payment_ratio,ratio_payment_coverage,ratio_medicare_pct_paid,region_avg_medicare_paid,Medicare Paid Ratio
0,91,32963.07,5777.24,4763.73,1013.51,5.705678,11.137473,0.175264,0.824568,68.000000,...,1.338235,0.908445,1.037810,1.038763,1.033356,0.691641,1.128919,5.311252,0.823783,1.000954
1,14,15131.85,5787.57,4976.71,810.86,2.614543,57.918571,0.382476,0.859896,27.461538,...,0.509804,0.376065,1.008540,1.080948,0.714704,1.274242,1.873207,4.211412,0.804224,1.069224
2,24,37560.37,5434.95,4453.79,981.16,6.910895,40.881667,0.144699,0.819472,27.461538,...,0.873950,0.933470,0.947092,0.967369,0.864809,0.899420,0.708675,4.013431,0.804224,1.018959
3,25,13998.28,5417.56,4129.16,1288.40,2.583872,51.536000,0.387016,0.762181,27.461538,...,0.910364,0.347893,0.944062,0.896859,1.135614,1.133822,1.895443,3.732842,0.804224,0.947721
4,18,31633.27,5658.33,4851.44,806.89,5.590566,44.827222,0.178873,0.857398,27.461538,...,0.655462,0.786167,0.986018,1.053739,0.711205,0.986225,0.876044,4.199175,0.804224,1.066118


In [9]:
from pyod.models.lof import LOF
from pyod.utils.utility import standardizer
from pyod.models.combination import aom, moa, average, maximization

In [11]:
def correlation(dataset, threshold):
    """
    Remove columns that do exceeed correlation threshold
    """
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset
    return dataset

In [10]:
from sklearn.preprocessing import StandardScaler

# First need to standarize the numbers:
scal_num_dat = StandardScaler().fit_transform(num_dat)
# Get the column names for later
scal_num_dat_name = num_dat.columns

scal_num_dat = pd.DataFrame(scal_num_dat, columns= scal_num_dat_name)

In [15]:
new_dat = correlation(scal_num_dat, threshold= 0.8)
print(f"There are {len(new_dat.columns)} of columns from the remaining data")

There are 21 of columns from the remaining data


In [35]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(scal_num_dat,test_size=0.3)

#train_split_dim = scal_num_dat.shape[0] * 0.6
#train_data = scal_num_dat.loc[1:train_split_dim]
#test_data = scal_num_dat[train_split_dim:]

In [36]:
print(f"There are {train.shape[0]} of rows in train and {test.shape[0]} of rows in test")

There are 114145 of rows in train and 48920 of rows in test



### Local Outlier Factor:

Measures the local deviation of density of a given sample with respect to its neighbors. It is local in that the anomaly score depends on how isolated the object is with respect to the surrounding neighborhood. More precisely, the locality is given by k-nearest neighbors, whose distance is used to estimate the local density. By comparing the local density of a sample to the local densities of its neighbors, one can identify samples that have a substantially lower density than their neighbors. These are considered outliers.

There are several ways to tune the parameters of a LOF, and understanding the pros and cons of each approach, an in-depth understanding of the data produces the best results. After experimenting with a few parameters, I realize what parameters are a good fit for my data.

LOF requires to estimate the proportion of the outlier that's in your data, calculates the variables that meet this threshold. This approach is fundamentally different from the previous methods I've experimented with. The default parameters are 0.1 for LOF; based on my guess, I'm assuming that less than 10% of providers are committing fraud, and I've decided to that further.



#### Model 1

Consistent the other models I've analyzed, average covered charges are higher in the outlier category by 2.36x, and out of pocket payment is higher for the outlier categories. At first, it didn't make sense to me that it would be that high for outliers; however, after giving it, some thought it occurred to me that for a hospital to be reimbursed by Medicare, there are numerous compensation and verification hurdles that the hospitals must meet. Those hurdles are costly to the hospital provider, the process can sometimes take months, and there's no guarantee that the hospital will be reimbursed. For the procedures that are more expensive, it represents a huge risk from the providers' standpoint. That risk can be a great incentive for providers to be persuaded to commit fraud.

I also see that the providers' labels as outliers also have twice as many traffic as normal. Combined with the point stated earlier, There's a massive revenue collection risk, and cash flow delays that Medicare costs those providers.Given that the average medicare % paid is around the same for both th.e classes, my assumption is that since the providers overcharge for certain procedures, and reduces their risk by collecting a higher percentage upfront from customers. Those procedures tend to be an average more expensive.

The first models produced: 1957 outliers, and providers labels as outliers follows some of the same patterns that I've seen in the previous examples.

When comparing average covered charges with out of pocket payments, it's evident that the normal category has a defined range, Max covered charges of 400,000 and out of pocket payments of 15,000. The outlier category contains a higher variance. For those variables that overlap, it indicates that there are other parameters that separate them.

There are clear boundaries for the medicare % paid for the normal values, and medicare % paid to seem to be an import factor to distinguish between outliers and normal.


In [ ]:
test_copy = test.copy()

In [38]:
model_1 = LOF(n_neighbors= 200, metric= 'euclidean', contamination= 0.04)
model_1.fit(train)

LOF(algorithm='auto', contamination=0.04, leaf_size=30, metric='euclidean',
  metric_params=None, n_jobs=1, n_neighbors=200, p=2)

In [40]:
test_copy['LOF_1'] = model_1.fit_predict(test)
test_copy['LOF_1'] = outlier_detection(test_copy['LOF_1'])
test_copy['LOF_1'].value_counts()

NameError: name 'test_copy' is not defined